In [ ]:
%load_ext autoreload
%autoreload 2

# Continuous Control

---

In this notebook, we solve the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

The algorithm used is [proximal policy optimisation](https://arxiv.org/abs/1707.06347) with [generalised advantage estimation](https://arxiv.org/abs/1506.02438).

### Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### Imports

In [ ]:
import torch    
import torch.autograd as autograd         
import torch.nn as nn                     
import torch.nn.functional as F           
import torch.optim as optim               
import torch.distributions

import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer

import streamz
import streamz.dataframe

import pdb, gym, collections

hv.extension('bokeh')

In [ ]:
# Local imports
from ppo_agent import PPOAgent

### Third-party code

In [ ]:
# Useful function from Shangton Zhang's awesome repo
# https://github.com/ShangtongZhang/DeepRL
def random_sample(indices, batch_size):
    indices = np.asarray(np.random.permutation(indices))
    batches = indices[:len(indices) // batch_size * batch_size].reshape(-1, batch_size)
    for batch in batches:
        yield batch
    r = len(indices) % batch_size
    if r:
        yield indices[-r:]

### Training setup

We use holoviews to visualise the training progress in real-time.

In [ ]:
# Setup training progress output
def init_training_progress():

    max_length = 1000000
    rolling_size = 100

    training_stream = streamz.Stream()
    example = pd.DataFrame({'x': [0]}, index=[0])

    training_sdf = streamz.dataframe.DataFrame(training_stream, example=example)

    training_raw_buffer = Buffer(training_sdf, length=max_length)
    training_smooth_buffer = Buffer(training_sdf.x.rolling(rolling_size).median())

    training_raw_dmap = hv.DynamicMap(hv.Curve, streams=[training_raw_buffer]).relabel('raw')
    training_smooth_dmap = hv.DynamicMap(hv.Curve, streams=[training_smooth_buffer]).relabel('smooth')
    
    return training_stream, training_raw_dmap, training_smooth_dmap

In [ ]:
def play_episode_and_learn(env, agent, max_episode_length=1000):
    
    # Run concurrent episode on all environments
    env_info = env.reset(train_mode=True)[brain_name]
    state = torch.from_numpy(env_info.vector_observations).float()
    
    # Start episode
    agent.start_episode()
    
    for _ in range(max_episode_length):    
        
        # Calculate actions for all envs
        action = agent.act(state)
        
        # Run through the envs in parallel
        env_info = env.step(action.numpy())[brain_name]
        next_state = torch.from_numpy(env_info.vector_observations).float()
        reward = torch.tensor(env_info.rewards).float()
        done = env_info.local_done
        
        # Record the experience tuple with the agent
        agent.record_outcome(reward)
        
        # Advance
        state = next_state

        # We want rectangular input to network, so if any finish early we finish all early
        if np.any(done):
            print("Someone finished")
            break
    
    # Finalise episode
    agent.finish_episode()

### Training the agent

The environment is considered solved when we reach an average score of 30 over 100 succesive episodes.

In [ ]:
# Create policy and agent
state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size
agent = PPOAgent(state_size, action_size, lr=3e-4, rollout_length=500, gamma=0.99, minibatch_size=64)
episode_index = 0

In [ ]:
# Display live training progress
stream, smooth, raw = init_training_progress()
layout = (smooth * raw)
layout.opts(
    opts.Curve(width=900, height=300, show_grid=True, tools=['hover'])
)

In [ ]:
# Train
num_episodes = 300
max_episode_length = 1000
last_100_average_returns = collections.deque(maxlen=100)

for _ in range(num_episodes):
    
    play_episode_and_learn(env, agent, max_episode_length=max_episode_length)
    
    average_episode_return = (agent.average_episode_return() / max_episode_length) * 1000
    last_100_average_returns.append(average_episode_return)
    score = np.mean(last_100_average_returns)
    print("Average return over last 100 episodes {}".format(score), end="\r")
    
    stream.emit( pd.DataFrame({'x': average_episode_return}, index=[episode_index]) )

    if score > 30:
        print("Reached target score of 30 in {} episodes".format(episode_index))
        break
    
    episode_index += 1

### Save/load model

In [ ]:
# Save model
torch.save(agent.policy, 'agent.policy.model')
torch.save(agent.value_estimator, 'agent.value_estimator.model')

In [ ]:
# Load model
agent.policy = torch.load('agent.policy.model')
agent.policy.eval()
agent.value_estimator = torch.load('agent.value_estimator.model')
agent.value_estimator.eval()

### Evaluate the model

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]      
num_agents = len(env_info.agents)
states = torch.from_numpy(env_info.vector_observations).float()                  
while True:
    actions = agent.act(states).numpy()
    actions = np.clip(actions, -1, 1)                  
    env_info = env.step(actions)[brain_name]           
    next_states = torch.from_numpy(env_info.vector_observations).float()
    dones = env_info.local_done                        
    states = next_states                               
    if np.any(dones):                                  
        break

In [ ]:
# env.close()